In [1]:
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle

Using TensorFlow backend.


In [42]:
from keras.preprocessing.image import load_img,img_to_array
image = load_img('FGNET/images/001A02.jpg')

In [13]:
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input

# Load images
Define a function which loops over all images in a directory, load these images into python and preprocess them to be able to use them in vgg16

In [14]:
# Try loading all images
def readImages(path):
    image_info = []
    for filename in os.listdir(path):
        if filename.endswith(".JPG"):
            img = load_img(os.path.join(path,filename),target_size = (224,224))
            img_info = os.path.basename(filename)[:-4].split("A")
            img_subject = img_info[0]
            img_age = img_info[1]
            img = img_to_array(img)
            img = img.reshape((1,img.shape[0],img.shape[1],img.shape[2]))
            img = preprocess_input(img)
            
            image_info.append((img_subject,img_age,img))
    return(image_info)
            

In [15]:
%%time
img_FGNET = readImages('FGNET/images/')

Wall time: 4.52 s


In [16]:
with open('FGNET_data.txt','wb') as fp:
    pickle.dump(img_FGNET,fp)

In [17]:
len(img_FGNET)

1002

In [18]:
from keras.applications.vgg16 import VGG16
from keras.layers import Input

In [19]:
from keras.layers import Dense, Flatten

In [20]:
conv_base = VGG16(weights = 'imagenet', include_top = False, input_shape = (224,224,3))
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [21]:
from keras.applications import vgg16
from keras import *

In [22]:
from keras import Sequential
from keras.layers import Dense, MaxPooling2D, Conv2D, Flatten, Dropout

# Exclude the layers above from training as they have been trained alreaday
for layer in conv_base.layers:
    layer.trainable = False
    
# Add output layers
x = Flatten()(conv_base.output)
x = Dense(1, activation = 'softmax')(x)

cnn_model = Model(inputs = conv_base.input,outputs = x)
cnn_model.compile(loss= 'binary_crossentropy',optimizer = 'rmsprop', metrics = ['accuracy'])
cnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [25]:
import random

In [26]:
%%time
img_FGNET = pickle.load( open("FGNET_data.txt","rb"))
img_FGNET_shuffled = random.sample(img_FGNET,len(img_FGNET))
train_x = []
train_y = []
for i in img_FGNET_shuffled[:800]:
    train_x.append(i[2])
    int_y = str(i[1])
    if type(int_y) is str:
        int_y = int_y[:2]
        int_y = int(int_y)
    if int_y > 18:
        _y = 1
    else:
        _y = 0
    train_y.append(_y)

Wall time: 398 ms


In [27]:
val_x = []
val_y = []
for i in img_FGNET_shuffled[800:]:
    val_x.append(i[2])
    int_y = str(i[1])
    if type(int_y) is str:
        int_y = int_y[:2]
        int_y = int(int_y)
    if int_y > 18:
        _y = 1
    else:
        _y = 0
    val_y.append(_y)

In [28]:
train_x = np.concatenate(train_x, axis = 0)

In [ ]:
cnn_model.fit(train_x,train_y, steps_per_epoch = 50, epochs = 10)

Epoch 1/10
